### `__missing__` 을 사용해 키에 따라 다른 디폴트 값을 생성하는 방법을 알아두라
* 디폴트 값은 만든 과정에서 계산 비용이 높거나 예외가 발생할 수 있는 상황에서는 setdefault를 사용하지 말것
* defaultdict에 전달되는 함수는 인자를 받지 않는다. 그래서 접근한 키값에 맞는 디폴트 값을 생성하는 것은 불가능하다.
* 디폴트 키를 만들때 어떤 키를 사용했는지 알아야 된다면, dict를 상속받고 __missing__ 을 오버라이딩한다.

### 선행 지식
* 대입식
* try/except/else 문
* dictionary의 setdefault 함수
* file open 함수
* `__missing__()` 오버라이딩의 정확한 의미

setdefault 나 defaultdict 는 키가 없는 경우 처리를 잘해준다. 그러나 이를 사용하는
 것이 적당하지 않은 경우가 있다.
SNS 프로필 사진을 관리하는 프로그램을 작성한다고 가정하자.

파일 핸들이 이미 딕셔너리안에 있으면 이 코드는 딕셔너리를 단 한번만 읽는다. 파일 핸들이 없으면 handle을 오픈한 다음에 그 핸들을 pictures 딕셔너리에 대입한다.


In [15]:
pictures= {}
path = 'test.png'

if (handle:=pictures.get(path)) is None:
    try:
        print(f'{path}를 오픈합니다.')
        handle = open(path, 'a+b')
    except OSError:
        print(f'경로를 열 수 없습니다: {path}')
        raise
    else:
        pictures[path] = handle


handle.seek(0)
image_data = handle.read()

test.png를 오픈합니다.


대입식 후에 open 하는 코드를 한 개의 코드로 바꿔서 효율적인 코드로 바꾸기 위해 다음과 같은 방식도 생각해 볼 수 있다.

In [16]:
try:
    handle = pictures.setdefault(path, open(path, 'a+b'))
except OSError:
    print(f'경로를 열 수 없습니다.: {path}')
    raise
else:
    handle.seek(0)
    image_data = handle.read()

위의 코드는 여러가지 문제점들을 안고 있다. open이 딕셔너리에 경로가 있든지 없든지 상관없이 항상 호출된다. 즉 open이 예외를 던질수도 있다. 그러나 이러한 예외가 setdefault 예외와 구분하지 못할 수 도 있다.

defaultdict를 이용해 구현해 보면 다음과 같다.
그런데 문제는 defaultdict 생성자에 전달한 함수는 인자를 받을 수 없다.

In [32]:
from collections import defaultdict

def open_picture(profile_path):
    try:
        return open(profile_path, 'a+b')
    except OSError:
        print(f'경로를 열 수 없습니다.: {profile_path}')
        raise

defaultdict(open_picture)
handle = pictures[path]
handle.seek(0)
image_data = handle.read()
print(image_data)

TypeError: open_picture() missing 1 required positional argument: 'profile_path'

위의 문제를 해결하기 위해 `__missing__` 특별 매서드를 구현하고 키가 없는 경우를 처리하는 로직을 구현한다. 아래에서 보는 바와 같이 pictures[key]를 호출했는데 key가 없다면`__missing__` 함수가 호출이 되고, 그 안에서 open_picutre 도우미 함수가 호출이 된다. 그래서 key에 해당하는 value가 생성이 되고, 그 value를 key에 대입하면서 value를 return해 준다. 그 이후 다시 호출하면 이미 key가 만들어져 있으므로 missing은 호출되지 않는다.

In [35]:
 class Pictures(dict):
    def __missing__(self, key):
        value = open_picture(key)
        self[key]= value
        return value

pictures = Pictures()
handle = pictures[path]
handle.seek(0)
image_data = handle.read()